In [58]:
import scvelo as scv
import numpy as np
import scanpy as sc

In [ ]:
SEED = 0
n_genes = 100

In [60]:
adata = scv.datasets.simulation(n_obs=1500,
                                n_vars=n_genes,
                                alpha=5,
                                beta=0.3,
                                gamma=0.5, 
                                random_seed=SEED)
adata

AnnData object with n_obs × n_vars = 1500 × 500
    obs: 'true_t'
    var: 'true_t_', 'true_alpha', 'true_beta', 'true_gamma', 'true_scaling'
    layers: 'unspliced', 'spliced'

In [61]:
spliced = adata.layers["spliced"].astype(np.float64)
unspliced = adata.layers["unspliced"].astype(np.float64)
beta = adata.var['true_beta'].values.astype(np.float64)
gamma = adata.var['true_gamma'].values.astype(np.float64)

In [62]:
true_velocity = beta * unspliced - gamma * spliced
adata.layers["true_velocity"] = true_velocity

In [63]:
scv.pp.remove_duplicate_cells(adata)
adata.layers['raw_spliced'] = np.rint(adata.layers['spliced']).astype(int)
adata.layers['raw_unspliced'] = np.rint(adata.layers['unspliced']).astype(int)
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
if adata.n_vars < 2000:
    sc.pp.highly_variable_genes(adata, n_top_genes=adata.n_vars, subset=True)
scv.pp.moments(adata, n_neighbors=30, n_pcs=30)
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=30)
sc.tl.umap(adata, random_state=SEED)
adata.obs['u_lib_size_raw'] = adata.layers['raw_unspliced'].sum(-1) 
adata.obs['s_lib_size_raw'] = adata.layers['raw_spliced'].sum(-1)

Skip filtering by dispersion since number of variables are less than `n_top_genes`.
Logarithmized X.
computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities


d:\softwares\anaconda3\envs\scvelo_py39\lib\site-packages\scvelo\preprocessing\utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)
C:\Users\18307\AppData\Local\Temp\ipykernel_36340\2209009637.py:7: DeprecationWarning: Automatic neighbor calculation is deprecated since scvelo==0.4.0 and will be removed in a future version of scVelo. Please compute neighbors first with Scanpy.
  scv.pp.moments(adata, n_neighbors=30, n_pcs=30)
d:\softwares\anaconda3\envs\scvelo_py39\lib\site-packages\scvelo\preprocessing\moments.py:71: DeprecationWarning: `neighbors` is deprecated since scvelo==0.4.0 and will be removed in a future version of scVelo. Please compute neighbors with Scanpy.
  neighbors(


    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [64]:
adata.write_h5ad(f"ode{n_genes}/sim_ode_{SEED}.h5ad")